In [1]:
import numpy as np
import math
import re
import time

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

2024-12-22 06:43:17.660854: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-22 06:43:17.799170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734849797.863077    1870 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734849797.880099    1870 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 06:43:18.018714: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
with open("data/europarl-v7.de-en.en", mode='r', encoding="utf-8") as f:
    text_en = f.read()

with open("data/europarl-v7.de-en.de", mode='r', encoding="utf-8") as f:
    text_de = f.read()

print(text_en[:50])
print(text_de[:50])
     


Resumption of the session
I declare resumed the se
Wiederaufnahme der Sitzungsperiode
Ich erkläre die


In [4]:
with open("data/nonbreaking_prefix.en", mode='r', encoding="utf-8") as f:
    non_breaking_prefix_en = f.read()

with open("data/nonbreaking_prefix.de", mode='r', encoding="utf-8") as f:
    non_breaking_prefix_de = f.read()
print(non_breaking_prefix_en[:5])
print(non_breaking_prefix_de[:5])

A
B
C
A
B
C


In [5]:
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref.lower() + '.' for pref in non_breaking_prefix_en]
non_breaking_prefix_de = non_breaking_prefix_de.split("\n")
non_breaking_prefix_de = [' ' + pref.lower() + '.' for pref in non_breaking_prefix_de]
print(non_breaking_prefix_en[:5])
print(non_breaking_prefix_de[:5])

[' a.', ' b.', ' c.', ' d.', ' e.']
[' a.', ' b.', ' c.', ' d.', ' e.']


In [6]:
for prefix in non_breaking_prefix_en:
    text_en = text_en.replace(prefix, prefix + '###')
text_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_en)
text_en = re.sub(r"\.###", '', text_en)
text_en = re.sub(r" +", ' ', text_en)
text_en = text_en.replace('###', ' ')

text_en = text_en.split("\n")

for prefix in non_breaking_prefix_de:
    text_de = text_de.replace(prefix, prefix + '###')
text_de = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_de)
text_de = re.sub(r"\.###", '', text_de)
text_de = re.sub(r" +", ' ', text_de)
text_de = text_de.replace('###', ' ')

text_de = text_de.split("\n")

In [7]:
import tensorflow_datasets as tfds

tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_en, target_vocab_size=8000
)
tokenizer_de = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_de, target_vocab_size=8000
)

In [8]:


VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_DE = tokenizer_de.vocab_size + 2

inputs = [
    [VOCAB_SIZE_EN - 2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN - 1]
    for sentence in text_en
]
outputs = [
    [VOCAB_SIZE_DE - 2] + tokenizer_de.encode(sentence) + [VOCAB_SIZE_DE - 1]
    for sentence in text_de
]

print("Sample input sentences:")
for i in range(3):
    print(f"Original: {text_en[i]}")
    print(f"Encoded: {inputs[i]}")
    print()

print("Sample output sentences:")
for i in range(3):
    print(f"Original: {text_de[i]}")
    print(f"Encoded: {outputs[i]}")
    print()


Sample input sentences:
Original: Resumption of the session
Encoded: [8025, 2483, 938, 2449, 3, 1, 2544, 8026]

Original: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Encoded: [8025, 11, 5674, 7238, 1, 2641, 3, 1, 24, 68, 1879, 4817, 52, 12, 6705, 7801, 4087, 1489, 1905, 2, 5, 11, 33, 57, 413, 396, 4, 311, 55, 7, 1735, 79, 322, 6, 1, 209, 8, 55, 4322, 42, 7, 5302, 1661, 4725, 3966, 2055, 7815, 8026]

Original: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
Encoded: [8025, 947, 2043, 2, 21, 55, 27, 18, 1419, 7879, 2, 1, 3349, 2448, 7869, 185, 1775, 4403, 7085, 503, 1716, 7872, 87, 2266, 4, 4594, 3441, 2, 176, 1, 102, 6, 7, 213, 3, 103, 4229, 7, 2198, 3, 1045, 5040, 8, 2037, 115, 3349,

In [9]:
MAX_LENGTH = 20 # we will still have a lot of data with max len of 20

# this part, why we do it is a bit tricky, pay attention why we do it like this:
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]
# we remove in reversed because of shifting issuies when we start from begining
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

# same stuff for outputs>20
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

Input / output creation

In [10]:

inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)

In [11]:
import os
import tensorflow as tf

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Dataset settings
BATCH_SIZE = 64
BUFFER_SIZE = 20000  # how much data to keep

# Assuming `inputs` and `outputs` are already defined
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

# Cache dataset for better performance
dataset = dataset.cache()

# Shuffle and batch the dataset
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Prefetch to improve performance
dataset = dataset.prefetch(tf.data.AUTOTUNE)


2024-12-22 07:21:23.078852: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-12-22 07:21:23.079529: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: ip-10-192-12-28
2024-12-22 07:21:23.079673: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: ip-10-192-12-28
2024-12-22 07:21:23.079923: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 535.216.3
2024-12-22 07:21:23.079968: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 535.216.3
2024-12-22 07:21:23.079975: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 535.216.3


Postional Encoding

In [12]:
class PositionalEncoding(layers.Layer):
    
    def __init__(self):
        # this Positional Encoder we made it a child of the Layers so it has all
        # the properties that a layer has
        super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model):
        """
        :pos: (seq_len, 1) index of the word in sentence [0 to 19]
        :i: the dimensions of the embedding (glove dims 200) then-> [0 to 199]
        :d_model: the size (dimension) of the embeded (e.g. glove size 200)
        :return: (seq_len, d_model) why? we are getting the encoding of the
                every positions vs every one of the dimensions of that word
        """
        angles = 1 / np.power(10000., (2*(i//2))/np.float32(d_model))
        return pos * angles # dim: (seq_len, d_model)

    def call(self, inputs):
        # input.shape = [batch_size, multihead_size(sz=8), each word (pos), that words embedding]
        # keep in mind we DONT change the values of the input considering 
        # their positions, we just get the dims from input and calculate
        # pos encoding totally seperatly and stack them at the end
        seq_length = inputs.shape.as_list()[-2] # basically the pos
        d_model = inputs.shape.as_list()[-1] # basically the embedded values
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],
                                 np.arange(d_model)[np.newaxis, :],
                                 d_model)
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        # we do this because it has a [batch] dimension at the begining we add
        # it. why? because inputs and the encodings need to be same dims so we
        # make newaxis which it doesnt put 0's.... it copies those same dims for
        # all the batches...
        pos_encoding = angles[np.newaxis, ...]
        # now we need to return both the inputs and their pos_encodings
        # but we have pos_encoding in np so we make them tf
        return inputs + tf.cast(pos_encoding, tf.float32)
     

Attention

In [13]:
def scaled_dot_product_attention(queries, keys, values, mask):
    # Q*K will be [output_len, d_model] * [d_model, input_len] which both are 20
    # for both english and french
    # the transpose_b=True makes keys turn to keys.T
    # each of them are this dim: [batch_size, nb_proj, seq_len, d_proj]
    # so with transpose it become: [a,b,c,d] * []
    product = tf.matmul(queries, keys, transpose_b=True)
    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32) # makes the dim_num float
    scaled_product = product / tf.math.sqrt(keys_dim) # scales it (formula stuff)

    # because this mask as the paper said is optional to prevent the program
    # from seeing the feauture. why? because when we backprop then they will
    # consider the stuff in front of them so to stop this we add -1e9 to them
    # so after softmax the probabilities become 0 for them
    if mask is not None:
        scaled_product += (mask * -1e9)
    
    # we apply the softmax along the last axis because we want their sum to be 1
    # scaled_product = [output_len, input_len] -> softmax on input_len so
    # basically we are keeping in_len the same but finding the probs for out_len
    # so for every ins what is the prob of each of the outs
    # (e.g. ith input, the probs [0.3,0.7] of the outs)
    probs = tf.nn.softmax(scaled_product, axis=-1)

    # attention = [output_len, input_len] * [input_len, d_model] = [output_len, d_model]
    # so now we have d_model weights for each of the output words which we will
    # feed to forwards to see their prediction for each of the out_lens
    attention = tf.matmul(probs, values)

    return attention
     

Multi-Head Attention


In [14]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, nb_proj):
        """
        :nb_proj: the number of projections for the multihead
        """
        super(MultiHeadAttention, self).__init__()
        self.nb_proj = nb_proj
    
    # this is the same as init but it happens when we USE the object for the
    # first time, in init it was called when we CREATED the object
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        # we wanna make sure they are divisible
        assert self.d_model % self.nb_proj == 0
        # we use 2 slashes to make it integer
        self.d_proj = self.d_model // self.nb_proj

        self.query_lin = layers.Dense(self.d_model)
        self.key_lin = layers.Dense(self.d_model)
        self.value_lin = layers.Dense(self.d_model)
        self.final_lin = layers.Dense(self.d_model)
    
    def split_proj(self, inputs, batch_size):
        """
        :inputs: [batch_size, seq_len(20), d_model(prev layer dim)]

        :return: 
            dims = [batch_size, nb_proj, seq_len, d_proj]
            nb_proj here is like channels in cnn
            we basically split the d_model to nb_proj*d_proj so d_proj is
            found by doing d_model/nb_proj
        """
        new_shape = (batch_size, -1, self.nb_proj, self.d_proj)
        # here we will get: [batch_sz, seq_len, nb_proj, d_proj]

        splited_inputs = tf.reshape(inputs, shape=new_shape)

        # so we need to reshape it to: [batch_size, nb_proj, seq_len, d_proj]
        return tf.transpose(splited_inputs, perm=[0, 2, 1, 3])
    
    def call(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]

        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(values)

        # now we split each of them to make projs
        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)

        # each of the q,k,v are [batch_size, nb_proj, seq_len, d_proj]
        attention = scaled_dot_product_attention(queries, keys, values, mask)

        # now we will reverse the splits we did above: reshape + concat
        attention = tf.transpose(attention, perm=[0,2,1,3])
        # we have [batch_size, seq_len, nb_proj, d_proj] so now we concat 2, 3
        concat_attention = tf.reshape(attention, shape=(batch_size, -1, self.d_model))
        outputs = self.final_lin(concat_attention)
        return outputs

Encoder

In [15]:
class EncoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout):
        """
        :FFN_units:
            Feed forward network units: the number of units for the
            feed forward network that follows the attention in the encoder.
        :nb_proj: 
            The number of projections (e.g., 8 for multi-head attention).
        :dropout:
            The dropout rate (e.g., 0.3).
        """
        super(EncoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout

    def build(self, input_shape):
        """
        Initialize layer components.
        """
        self.d_model = input_shape[-1]
        # Build the multi-head attention component
        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

        self.dense_1 = layers.Dense(units=self.FFN_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model, activation="relu")
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, mask):
        """
        Performs forward pass.
        :inputs: Input tensor [batch_size, seq_len, d_model]
        :mask: Attention mask tensor
        """
        # Multi-head attention (query, key, value are all the same input)
        attention = self.multi_head_attention(inputs, inputs, inputs, mask)

        # Dropout + normalization after attention
        attention = self.dropout_1(attention)  # Dropout automatically handled by TensorFlow
        attention = self.norm_1(attention + inputs)  # Residual connection + normalization
        
        # Feed Forward Network (FFN)
        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs)  # Dropout automatically handled by TensorFlow
        outputs = self.norm_2(outputs + attention)  # Residual connection + normalization

        return outputs


In [16]:
class Encoder(layers.Layer):
    def __init__(self,
                 nb_encoding_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 vocab_size,
                 d_model,
                 name="encoder"):
        super(Encoder, self).__init__(name=name)
        self.nb_encoding_layers = nb_encoding_layers  # Number of encoder layers
        self.d_model = d_model  # The size of the output (e.g., glove(200))

        # Embedding and Positional Encoding
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)

        # Encoder layers
        self.enc_layers = [EncoderLayer(FFN_units, nb_proj, dropout)
                           for _ in range(self.nb_encoding_layers)]
    
    def call(self, inputs, mask):
        """
        Forward pass for the Encoder.
        """
        # Embedding layer
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))  # Scale embeddings
        
        # Add positional encoding
        outputs = self.pos_encoding(outputs)
        
        # Dropout before encoding layers
        outputs = self.dropout(outputs)  # Dropout will be applied automatically in training mode
        
        # Apply each encoder layer
        for i in range(self.nb_encoding_layers):
            outputs = self.enc_layers[i](outputs, mask)
        
        return outputs


Decoder


In [17]:
class DecoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout):
        super(DecoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout = dropout

    def build(self, input_shape):
        """
        Initialize the layer components.
        """
        self.d_model = input_shape[-1]

        # MHA 1 (Multi-head attention 1)
        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)
    
        # MHA 2 (Multi-head attention 2)
        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

        # Feed Forward Network (FFN)
        self.dense_1 = layers.Dense(units=self.FFN_units, activation='relu')
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_3 = layers.Dropout(rate=self.dropout)
        self.norm_3 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, enc_outputs, mask_1, mask_2):
        """
        Performs forward pass.
        :inputs: The decoder's input tensor.
        :enc_outputs: The encoder's output tensor.
        :mask_1: Mask for the first attention (self-attention).
        :mask_2: Mask for the second attention (encoder-decoder attention).
        """
        # First attention (self-attention)
        attention = self.multi_head_attention_1(inputs, inputs, inputs, mask_1)
        attention = self.dropout_1(attention)  # Dropout automatically handled by TensorFlow
        attention = self.norm_1(attention + inputs)  # Residual connection + normalization

        # Second attention (encoder-decoder attention)
        attention_2 = self.multi_head_attention_2(attention, enc_outputs, enc_outputs, mask_2)
        attention_2 = self.dropout_2(attention_2)  # Dropout automatically handled by TensorFlow
        attention_2 = self.norm_2(attention_2 + attention)  # Residual connection + normalization

        # Feed Forward Network (FFN)
        outputs = self.dense_1(attention_2)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_3(outputs)  # Dropout automatically handled by TensorFlow
        outputs = self.norm_3(outputs + attention_2)  # Residual connection + normalization

        return outputs


In [18]:
class Decoder(layers.Layer):
    def __init__(self,
                 nb_decoding_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 vocab_size,
                 d_model,
                 name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.nb_decoding_layers = nb_decoding_layers  # Number of decoder layers
        self.d_model = d_model  # The size of the output (e.g., glove(200))

        # Embedding and Positional Encoding
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)

        # Decoder layers
        self.dec_layers = [DecoderLayer(FFN_units, nb_proj, dropout)
                           for _ in range(nb_decoding_layers)]
    
    def call(self, inputs, enc_outputs, mask_1, mask_2):
        """
        Forward pass for the Decoder.
        """
        # Embedding layer
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))  # Scale embeddings
        
        # Add positional encoding
        outputs = self.pos_encoding(outputs)
        
        # Dropout before decoding layers
        outputs = self.dropout(outputs)  # Dropout will be applied automatically in training mode
        
        # Apply each decoder layer
        for i in range(self.nb_decoding_layers):
            outputs = self.dec_layers[i](outputs,
                                         enc_outputs,
                                         mask_1,
                                         mask_2)
        
        return outputs


Transformer


In [19]:
class Transformer(tf.keras.Model):
    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 name="transformer"):
        super(Transformer, self).__init__(name=name)

        # Initialize Encoder and Decoder
        self.encoder = Encoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout,
                               vocab_size_enc,
                               d_model)
        self.decoder = Decoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout,
                               vocab_size_dec,
                               d_model)

        # Final linear layer for output
        self.last_linear = layers.Dense(units=vocab_size_dec)

    def create_padding_mask(self, seq):
        """
        Creates a mask for padding tokens in the input sequence.
        """
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(self, seq):
        """
        Creates a look-ahead mask to prevent attending to future positions.
        """
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask

    def call(self, enc_inputs, dec_inputs):
        """
        Forward pass for the Transformer model.
        """
        # Encoder mask
        enc_mask = self.create_padding_mask(enc_inputs)

        # Decoder masks
        dec_mask_1 = tf.maximum(self.create_padding_mask(dec_inputs),
                                self.create_look_ahead_mask(dec_inputs))
        dec_mask_2 = self.create_padding_mask(enc_inputs)

        # Encoder forward pass
        enc_outputs = self.encoder(enc_inputs, enc_mask)

        # Decoder forward pass
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2)

        # Final linear layer
        outputs = self.last_linear(dec_outputs)

        return outputs


Testing masks to see their outputs

In [20]:
def create_padding_mask(seq):
    mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(seq):
    seq_len = tf.shape(seq)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    return look_ahead_mask

Testing

In [26]:
import tensorflow as tf

# Clear the current session to avoid clutter from previous models
tf.keras.backend.clear_session()

# Hyper-parameters:
EPOCHS =3
D_MODEL = 512  # 512 takes more time but has a lot better results
NB_LAYERS = 6   # 6
FFN_UNITS = 2048  # 2048
NB_PROJ = 8      # 8
DROPOUT = 0.1    # 0.1

# Instantiate the Transformer model with the specified hyperparameters
transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_DE,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout=DROPOUT)


loss

In [22]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')
def loss_function(target, pred):
    # so by using this mask we will get rid of all the losses that 
    # corrispond to 0 in our target (aka. y_true)
    mask = tf.math.logical_not(tf.math.equal(target, 0)) # [326, 4, 0] -> [1, 1, 0]
    loss_ = loss_object(target, pred) # we got the loss numbers (not their softmax probabilities)
    
    # make sure that both loss_ and mask have the same data type so we can mult them
    mask = tf.cast(mask, dtype=loss_.dtype)
    # we make the computed losses for 0's 0
    loss_ *= mask
    
    # compute the mean loss and return
    return tf.reduce_mean(loss_)

# keeps track of losses during training
train_loss = tf.keras.metrics.Mean(name="train_loss")
# keeps track of accs during training
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

optimizer

In [28]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    # Overriding __call__ method
    def __call__(self, step):
        # Ensure step is cast to float32 for compatibility
        step = tf.cast(step, tf.float32)
        
        # arg1 and arg2 calculations with proper casting
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        # Compute the learning rate using the formula from the paper
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# Example usage
learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)


clearing checkpoints

In [29]:
checkpoint_path = "./ckpt/"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# lets check if we already have a checkpoint
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint Restored...")

epochs

In [30]:
import time
import os
import tensorflow as tf

# Define directory for saving model
model_save_dir = "./model"

# Create the directory if it doesn't exist
os.makedirs(model_save_dir, exist_ok=True)

for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch + 1))
    start = time.time()

    # Reset the states of loss and accuracy metrics at the start of each epoch
    train_loss.reset_state()
    train_accuracy.reset_state()

    # Iterate over the dataset in batches
    for (batch_index, (enc_inputs, targets)) in enumerate(dataset):
        # Ensure correct data types
        enc_inputs = tf.cast(enc_inputs, dtype=tf.float32)
        dec_inputs = tf.cast(targets[:, :-1], dtype=tf.float32)  # Decoder inputs (all but the last token)
        dec_outputs_real = tf.cast(targets[:, 1:], dtype=tf.float32)  # Decoder outputs (all but the first token)

        print(f"Encoder input shape: {enc_inputs.shape}")
        print(f"Decoder input shape: {dec_inputs.shape}")

        with tf.GradientTape() as tape:
            try:
                # Ensure correct call to transformer
                predictions = transformer(enc_inputs, dec_inputs)  # Removed training=True argument
                loss = loss_function(dec_outputs_real, predictions)
            except Exception as e:
                # Catch exceptions and print the error message for debugging
                print(f"Error in transformer call: {e}")
                break  # Exit the loop if an error occurs

        # Calculate gradients and apply them using the optimizer
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

        # Update loss and accuracy metrics
        train_loss.update_state(loss)
        train_accuracy.update_state(dec_outputs_real, predictions)

        # Print loss and accuracy periodically (every 50 batches)
        if batch_index % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch + 1, batch_index, train_loss.result(), train_accuracy.result()))

    # Save the entire model (architecture, weights, optimizer state) as a single file
    model_save_path = os.path.join(model_save_dir, f"epoch_{epoch + 1}.keras")  # Add .keras extension
    transformer.save(model_save_path)  # Save the entire model
    print(f"Saved full model for epoch {epoch + 1} at {model_save_path}")

    # Print final loss and accuracy for the epoch
    print("Epoch {} Loss {:.4f} Accuracy {:.4f}".format(
        epoch + 1, train_loss.result(), train_accuracy.result()))

    # Print time taken for the epoch
    print("Time taken for epoch {}: {:.2f} seconds\n".format(epoch + 1, time.time() - start))


Start of epoch 1
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)


Epoch 1 Batch 0 Loss 6.1303 Accuracy 0.0000
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 

2024-12-22 14:58:28.732719: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Saved full model for epoch 1 at ./model/epoch_1.keras
Epoch 1 Loss 2.5146 Accuracy 0.2729
Time taken for epoch 1: 23233.63 seconds

Start of epoch 2
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Epoch 2 Batch 0 Loss 1.9130 Accuracy 0.3133
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 2

2024-12-22 21:25:54.393161: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Saved full model for epoch 2 at ./model/epoch_2.keras
Epoch 2 Loss 1.6743 Accuracy 0.3584
Time taken for epoch 2: 23245.63 seconds

Start of epoch 3
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Epoch 3 Batch 0 Loss 1.3520 Accuracy 0.4128
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 20)
Decoder input shape: (64, 19)
Encoder input shape: (64, 2

KeyboardInterrupt: 

In [31]:
class Transformer(tf.keras.Model):
    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 name="transformer"):
        super(Transformer, self).__init__(name=name)

        # Initialize Encoder and Decoder
        self.encoder = Encoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout,
                               vocab_size_enc,
                               d_model)
        self.decoder = Decoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout,
                               vocab_size_dec,
                               d_model)

        # Final linear layer for output
        self.last_linear = layers.Dense(units=vocab_size_dec)

    def create_padding_mask(self, seq):
        """
        Creates a mask for padding tokens in the input sequence.
        """
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(self, seq):
        """
        Creates a look-ahead mask to prevent attending to future positions.
        """
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask

    def call(self, enc_inputs, dec_inputs):
        """
        Forward pass for the Transformer model.
        """
        # Encoder mask
        enc_mask = self.create_padding_mask(enc_inputs)

        # Decoder masks
        dec_mask_1 = tf.maximum(self.create_padding_mask(dec_inputs),
                                self.create_look_ahead_mask(dec_inputs))
        dec_mask_2 = self.create_padding_mask(enc_inputs)

        # Encoder forward pass
        enc_outputs = self.encoder(enc_inputs, enc_mask)

        # Decoder forward pass
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2)

        # Final linear layer
        outputs = self.last_linear(dec_outputs)

        return outputs


evaluation

In [32]:
def evaluate(inp_sentence):
    # turn the sentence to the tokenizer_encoded format [hi, bye] -> [241, 6]
    inp_sentence = [VOCAB_SIZE_EN-2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    # expand dim on axis=0 to simulate the batch dimmension
    enc_input = tf.expand_dims(inp_sentence, axis=0)

    # let's make the ouput which starts with  and add that axis=0 for batch=0
    output = tf.expand_dims([VOCAB_SIZE_DE-2], axis=0)

    # the loop to predict the next word of output each time and output += it
    for _ in range(MAX_LENGTH):
        # we put false because we are not training so no dropout
        # predictions = [btch_sz=1, seq_len(output_so_far), vocav_sz_de(the 
        # softmax values of each word, the higher the number the higher the 
        # probability for that word)]
        predictions = transformer(enc_input, output)
        # we want to take the last word of this prediction
        prediction = predictions[:, -1:, :]
        # we do argmax to get the index of the most probable next word
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)

        # we reached the end of the sentence
        if predicted_id == VOCAB_SIZE_DE-1:
            return tf.squeeze(output, axis=0)
        
        # now we know add the new prediction to the last of the output
        output = tf.concat([output, predicted_id], axis=-1)
    
    #even if we didn't reach the end of the sentence we can't continue
    return tf.squeeze(output, axis=0)

In [33]:
def translate(sentence):
    output = evaluate(sentence).numpy()
    # get rid of  and  if they exist
    output = [i for i in output if i < VOCAB_SIZE_DE-2]
    # decode indexes to words e.g. [241, 6] -> [hi, bye] 
    predicted_sentence = tokenizer_de.decode(output)
    print("Input: {}".format(sentence))
    print("Predicted translation: {}".format(predicted_sentence))

In [46]:
inputs = input("enter the sentence or word you want to translate :")
translate(inputs)

Input: how are you?
Predicted translation: Wie sind Sie das tun?
